# Generative AI for Non-Binding Loan Term Sheet Creation
When creating loan agreements, Financial Instituions (e.g. Credit Unions, Commercial Banks, Brokerage Firms etc.) need to create [Term Sheets](https://www.investopedia.com/terms/t/termsheet.asp#:~:text=A%20term%20sheet%20is%20a%20nonbinding%20agreement%20outlining%20the%20basic,with%20capital%20to%20fund%20enterprises) ; a non-binding agreement that outlines the terms of a loan. Writing terms sheets can be laborious and complex, and can involve extracting different points of information from call transcripts, company documents, and other sources of information.

First, AI21 Contextual Answers is used to extract key information about the potential loan. This is then passed to a chain of Jurassic-2 Foundation Model calls, which will first create an initial draft term sheet, criticise it if it is missing any important sections, and then accordingly adjust it. The finalized term sheet is then generated in the final text box. This is schematically shown below as:

![Architecture](img/img_1.png)

In this example, we will generate a candidate term sheet between "ABC Ventures" the lender, and "XYZ Tech Solutions", which is seeking the loan. The input is an unstructured document, and the output will be a structured term-sheet



Import libraries and define key functions

In [32]:
import json
import os
os.environ["AI21_LOG_LEVEL"] = "DEBUG"
from ai21 import AI21Client
from ai21 import errors as ai21_errors
from ai21 import AI21Client, AI21APIError
from ai21.models import ChatMessage
import re
from docx import Document
from concurrent.futures import ThreadPoolExecutor, as_completed


#In order to speed up the answer to the questions, we will call AI21 Contextual Answers in Parallel
def call_ca_parallel(args):
    article, question, category = args
    response = client.answer.create(
        context=article,
        question=question
    )
    answer = response.answer if response.answer else "None"
    return category, answer

def get_answered_questions(user_input, questions):
    answered_questions = {}
    unanswered_questions = {}

    # Use ThreadPoolExecutor to parallelize the calls
    with ThreadPoolExecutor(max_workers=7) as executor:
        # Prepare a list of arguments for the call_ca_parallel function
        future_to_question = {executor.submit(call_ca_parallel, (user_input, q[category], category)): category for q in questions for category in q}

        for future in as_completed(future_to_question):
            # When a future is completed, get the results
            category, answer = future.result()
            if answer != "None":
                answered_questions[category] = answer
            else:
                unanswered_questions[category] = "None"

    return answered_questions, unanswered_questions


client = AI21Client()


Next we will define key questions that define a term sheet. Note that you can add your own questions or have Jurassic dynamically create new questions as well. For this notebook though, we will just use these 20 questions.

In [18]:
questions=[
    {"Investment Amount":"What is the investment amount being sought by the startup or company?"},
    {"Security Type":"What type of security is being offered to investors?"},
    {"Pre-Money Evaluation":"What is the pre-money valuation of the startup or company?"},
    {"Post-Money Evaluation":"What is the post-money valuation of the startup or company?"},
    {"Percentage Ownership":"What percentage of the company will each investor own after the investment?"},
    {"Anti-Dilution":"What are the terms of the anti-dilution policy?"},
    {"Divident Policy":"What is the company's dividend policy?"},
    {"Redemption Rights":"What redemption rights are available to shareholders?"},
    {"Liquidation Preference":"What are the liquidation preferences specified in the term sheet?"},
    {"Governance Rights":"What governance rights are granted to investors?"},
    {"Information Rights":"What information rights are granted to investors?"},
    {"Transfer Rights":"What are the transfer restrictions on shareholders?"},
    {"Drag-along Rights":"What are the terms of the drag-along rights?"},
    {"First Refusal":"What are the terms for the right of first refusal?"},
    {"Clawback":"What are the clawback terms?"},
    {"Vesting Terms":"What are the vesting terms for equity granted?"},
    {"Business Model":"What is business model of the company seeking the loan?"},
    {"Closing Conditions":"What are the closing conditions for the loan"},
    {"Collateral": "What collateral has the company put up for the loan?"},
    {"Disclaimers":"What disclaimers are mentioned?"},
    
]

## Create a draft Term Sheet
First we will:

* Get the answers to the questions that define a term sheet. We will keep track of questions that do not have an answer.

* Draft a term sheet based on the question/answer pairs. Note that this is not the final Term Sheet.

In [33]:

raw_notes=f'''In a negotiation between XYZ Tech Solutions, which provides IT software for financial companies, and ABC Ventures, the focus was on a $2,000,000 equity investment with clear, numerical benchmarks. XYZ Tech, with an EBITDA of $1.2M last year, argued for an $8M pre-money valuation. ABC's investment would shift this to a $10M post-money valuation, granting ABC a 20% stake. The investment terms were sharply defined: Series A Preferred Stock carrying an 8% annual dividend, contingent on XYZ's EBITDA growth of at least 12% per year. As security for a potential loan, XYZ has listed its office buildings and land holdings, which are worth $3,000,000.

The proposed term sheet included a full-ratchet anti-dilution clause to protect ABC against devaluation, governance rights including a board seat and veto power on expenditures over $250,000, and quarterly financial audits to ABC. A five-year redemption clause was agreed upon, allowing ABC to exit with a 12% premium if XYZ's EBITDA grows by 20% within the period. This conversation framed a deal focused on financial health, growth metrics, and strategic alignment, streamlining the partnership terms into a concise, numbers-driven term sheet.
'''

answered_questions, unanswered_questions = get_answered_questions(raw_notes, questions)

#keep track of unanswered questions

z=list(unanswered_questions.keys())
for i in range(0,len(z)):
    category=z[i]
    for j in range(0,len(questions)):
        category_2=list(questions[j].keys())[0]
        if category==category_2:
            unanswered_questions[category]=list(questions[j].values())[0]
list_of_unanswered_questions=list(unanswered_questions.values())


In [34]:
# Create a term sheet based on the answered questions
prompt=f'''
You are a lending term-sheet writer. Given input data, you will write a term sheet that clearly fills out each section. The term
sheet should be clear and succinct. Make sure it is has correct punctuation. 

An example format is:

Investment Amount: $XXX
Security Type: ABC
Pre-Money Evaluation: $XXX
Post-Money Evaluation: $XXX
Percentage Ownership: XX%
Anti-Dilution: TEXT
Dividend Policy: TEXT
Redemption Rights: TEXT
Governance Rights: TEXT
Information Rights: TEXT
Business Model: TEXT
Collateral: TEXT
____

Here is the raw information:

{answered_questions}

Term Sheet:
'''

messages = [
    # Notice the given role does not exist and will be the reason for the raised error
    ChatMessage(text=prompt, role="user"),
]

system=""

response = client.chat.create(
    model="j2-ultra",  # You can choose from various models like j2-light, j2-mid, j2-ultra
    messages=messages,
    max_tokens=1100,
    temperature=0.0,
    system=system
)

#get the actual text from the response
#generated_text = response.completions[0].data.text
#generated_text=response.outputs[0].text
generated_text=response.outputs[0].text

term_sheet=generated_text
print(term_sheet)

Investment Amount: $2,000,000
Security Type: XYZ Tech Solutions is offering land holdings and office buildings to investors as security for a potential loan.
Pre-Money Evaluation: $8,000,000
Post-Money Evaluation: $10,000,000
Percentage Ownership: 20%
Anti-Dilution: The terms of the anti-dilution policy include a full ratchet clause.
Dividend Policy: The company's dividend policy is to pay out 8% of its annual earnings as dividends to shareholders.
Redemption Rights: A five-year redemption clause was agreed upon, allowing ABC to exit with a 12% premium if XYZ's EBITDA grows by 20% within the period.
Governance Rights: Investors are granted governance rights, including a board seat and veto power on expenditures over $250,000.
Information Rights: Investors are granted quarterly financial audits, a full-ratchet anti-dilution clause, and a redemption clause.
Business Model: XYZ Tech Solutions provides IT software for financial companies.
Collateral: The company has put up its office build

## Audit the term sheet
Next we will use Jurassic to criticise the draft term sheet. Jurassic will look at the text and the list of missing questions; and generate a criticism of the term sheet based on the missing questions.

In [35]:
prompt=f'''
You are a lending term-sheet auditor for ABC Ventures. However, the term sheet notes do not discuss any of the following questions. I want you to write 2 paragraph an
analysis of how the term sheet can be improved to answer these questions. Make specific suggestions, listing all of the missing sections.

For each missing section, you are to give a one sentence discussion of what is missing from the term sheet. Each criticism should be seperated by a newline.


Unanswered Questions:
{unanswered_questions}


Loan Notes:
{raw_notes}



Assessment:
'''

messages = [
    # Notice the given role does not exist and will be the reason for the raised error
    ChatMessage(text=prompt, role="user"),
]

system=""
#print(prompt)
response = client.chat.create(
    model="j2-ultra",  # You can choose from various models like j2-light, j2-mid, j2-ultra
    messages=messages,
    max_tokens=400,
    temperature=0,
    system=system
)

#get the actual text from the response
# criticism = response.completions[0].data.text
criticism = response.outputs[0].text
print(criticism)

The term sheet notes do not discuss any of the liquidation preferences, drag-along rights, transfer restrictions on shareholders, right of first refusal, vesting terms, clawback terms, disclaimers, or closing conditions for the loan. To improve the term sheet, the following sections should be included:

1. Liquidation Preferences: The term sheet should specify the liquidation preferences for shareholders, including the order of distribution of proceeds in the event of a liquidation or bankruptcy.
2. Drag-along Rights: The term sheet should outline the terms of the drag-along rights, which allow certain shareholders to force other shareholders to sell their shares in the event of a sale or merger.
3. Transfer Restrictions on Shareholders: The term sheet should specify any transfer restrictions on shareholders, such as the requirement for shareholder approval or the right of first refusal.
4. Right of First Refusal: The term sheet should detail the terms for the right of first refusal, w

## Create an enhanced Term Sheet
Next, Jurassic will analyze both the draft term sheet created above, as well as the criticism, and create an enhanced term sheet, which includes new sections for the missing term sheet sections.

In [36]:
prompt=f'''
You are a lending term-sheet auditor for ABC Ventures. However, the term sheet below is missing sections. You are to generate an enhanced term sheet that includes
the missing sections, in the same format as the original term sheet. The list of missing sections is listed in the "Missing Sections component, including a description of what is missing. The new term sheet must follow the same format as the original term sheet. Do not make comments other than creating a new term sheet. The new term sheet must have one section for each of the criticisms. Do not respond to the criticism; just create a new term sheet.

If the term sheet is missing a section, you are expected to fill it in to the best of your ability. Do not just note that the section is missing. 

For example: 

Criticism:

Right of first refusal are not discussed

New clause to address criticism:

Right of First Refusal: ABC has the right to purchase shares from other shareholders before they are able to sell them to a third party.

Remember, must first include in full the original term sheet, and then add the extra clauses you wish to add. Each new clause should be 1 sentence long.
Example format:

Investment Amount: $XXX
Security Type: ABC
Pre-Money Evaluation: $XXX
Post-Money Evaluation: $XXX
Percentage Ownership: XX%
Anti-Dilution: TEXT
Dividend Policy: TEXT
Redemption Rights: TEXT
Governance Rights: TEXT
Information Rights: TEXT
Business Model: TEXT
Collateral: TEXT
Liquidation Preferences: [the  liquidation clause]
Vesting Terms: [the vesting clause]
Claw-back terms: [the terms]


 ________

Original Term Sheet:
{term_sheet}

Missing Sections:
{criticism}

 Enhanced Term Sheet:

'''
#print(prompt)

messages = [
    # Notice the given role does not exist and will be the reason for the raised error
    ChatMessage(text=prompt, role="user"),
]

system=""
response = client.chat.create(
    model="j2-ultra",  # You can choose from various models like j2-light, j2-mid, j2-ultra
    messages=messages,
    max_tokens=600,
    temperature=0,
    system=system
)

#get the actual text from the response
#new_term_sheet_1 = response.completions[0].data.text
new_term_sheet_1=response.outputs[0].text
print(new_term_sheet_1)

Investment Amount: $2,000,000
Security Type: XYZ Tech Solutions is offering land holdings and office buildings to investors as security for a potential loan.
Pre-Money Evaluation: $8,000,000
Post-Money Evaluation: $10,000,000
Percentage Ownership: 20%
Anti-Dilution: The terms of the anti-dilution policy include a full ratchet clause.
Dividend Policy: The company's dividend policy is to pay out 8% of its annual earnings as dividends to shareholders.
Redemption Rights: A five-year redemption clause was agreed upon, allowing ABC to exit with a 12% premium if XYZ's EBITDA grows by 20% within the period.
Governance Rights: Investors are granted governance rights, including a board seat and veto power on expenditures over $250,000.
Information Rights: Investors are granted quarterly financial audits, a full-ratchet anti-dilution clause, and a redemption clause.
Business Model: XYZ Tech Solutions provides IT software for financial companies.
Collateral: The company has put up its office build

## Save New Term Sheet as a document
Now we will save the term sheet as a .docx document. 

In [37]:
doc = Document()
doc.add_heading('Enhanced Term Sheet', level=1)

# You might need to preprocess the term sheet text to split it into paragraphs
# or apply any other formatting you want. This is a simple example:
term_sheet_lines = new_term_sheet_1.split('\n')
for line in term_sheet_lines:
    if line.strip() != '':
        doc.add_paragraph(line)

# Save the document
docx_file_path = 'enhanced_term_sheet.docx'
doc.save(docx_file_path)

The output should look like this (although note that your specific results may vary). Note that the "Enhanced" term sheet includes the terms within the notes, as well as Jurassic's attempt to fill in the missing sections:

![Architecture](img/img_2.png)